In [12]:
import numpy as np
import pylab as pl
from scipy.stats import norm

In [64]:
dat = np.loadtxt('input_ex3.dat')

label=['ID','SS-IN','SED-IN','COND-IN','SS-OUT','SED-OUT','COND-OUT','STAT']
out=['OK','set','sol']
nout=3
nfeatures=len(label)-2 

In order to construct a proper Naives model, we need the prior of the target levels, i.e. $P\Big[t_j\Big]$ and the conditional probability of the features given the target levels $P\Big[f_i~|~t_j\Big]$. First, lets compte the priors:

In [53]:
priors = np.zeros(nout)

for i in range(len(priors)):
    priors[i] =  dat[:,-1].tolist().count(i+1)
    
priors /=np.sum(priors)
print(priors)

[ 0.30769231  0.38461538  0.30769231]


Now, let us compute the conditional pdf. Following the exercice, we will assume that all $P\Big[f_i~|~t_j\Big]$ are normally distributed. We thus have to determine $N_{features}\times N_{target}$ mean and standard deviations that characterized these pdf:

In [66]:
mean = np.zeros((nout,nfeatures))
std  = np.zeros((nout,nfeatures))

for j in range(nout):
    idx = np.where(dat[:,-1]==j+1)
    for i in range(nfeatures):
        mean[j,i] = np.mean(dat[idx,i+1])
        std[j,i]  = np.std(dat[idx,i+1])
        print('P('+label[i+1]+'|'+out[j]+'  = N(m='+str(mean[j,i])+',sig='+str(std[j,i])+')')
print (mean)
print(std)

P(SS-IN|OK  = N(m=189.0,sig=39.3319208786)
P(SED-IN|OK  = N(m=3.125,sig=0.216506350946)
P(COND-IN|OK  = N(m=1860.5,sig=321.64382475)
P(SS-OUT|OK  = N(m=18.0,sig=5.24404424085)
P(SED-OUT|OK  = N(m=0.054,sig=0.0843534231671)
P(COND-OUT|OK  = N(m=2036.0,sig=460.890984941)
P(SS-IN|set  = N(m=200.8,sig=49.30882274)
P(SED-IN|set  = N(m=4.4,sig=1.59373774505)
P(COND-IN|set  = N(m=1251.2,sig=103.971919286)
P(SS-OUT|set  = N(m=98.0,sig=20.9093280619)
P(SED-OUT|set  = N(m=1.018,sig=1.3654654884)
P(COND-OUT|set  = N(m=1372.0,sig=127.525683688)
P(SS-IN|sol  = N(m=1301.0,sig=420.403377722)
P(SED-IN|sol  = N(m=32.5,sig=10.3561575886)
P(COND-IN|sol  = N(m=1621.0,sig=392.342325017)
P(SS-OUT|sol  = N(m=49.1,sig=32.6975534253)
P(SED-OUT|sol  = N(m=1293.0,sig=373.214415584)
P(COND-OUT|sol  = N(m=832.85,sig=829.922738874)
[[  1.89000000e+02   3.12500000e+00   1.86050000e+03   1.80000000e+01
    5.40000000e-02   2.03600000e+03]
 [  2.00800000e+02   4.40000000e+00   1.25120000e+03   9.80000000e+01
    1.018

The model has been built (youpie) and we can now make predictions out of it. Our purpose is to determine what is the status of the water treatment given some features. For this, we use the Naives Bayes Model. We introduce the ``Model'' of a target given some features: $M\Big[t_j ~| ~ \mathbf f\Big]=\prod_{i=1}^{N_{features}}P\Big[f_i ~|~t_j\Big]P[t_j]$ (with $\mathbf f$ a set of features). This $M\Big[t_j ~| ~ \mathbf f\Big]\propto P\Big[t_j ~| ~ \mathbf f\Big]$ up to a constant that does not depend on the target levels (and under the assumption of conditional independence). The Naive Bayes model prediction is given by the target level which gives the maximal $M\Big[t_j ~| ~ \mathbf f\Big]$ (MAP estimator).

In conclusion, given a set of features $\mathbf f$, we need to compute $M\Big[t_j ~| ~ \mathbf f\Big]$ for the 3 target levels and choose the one which is maximal.
In the exercice 3, the set of features observed is given by
- SS-IN    = 222
- SED-IN   = 4.5
- COND-IN  = 1518
- SS-OUT   = 74
- SED-OUT  = 0.25
- COND-OUT = 1642

In [62]:
x = np.array([222,4.5,1518,74,0.25,1642])
M = priors.copy()

for j in range(nout):
    for i in range(nfeatures):
        print(j,i,x[i],mean[j,i],std[j,i],norm.pdf(x[i],mean[j,i],std[j,i]))
        M[j] *= norm.pdf(x[i],mean[j,i],std[j,i])

print(M)
print(M/np.sum(M))

0 0 222.0 189.0 39.3319208786 0.00713354825875
0 1 4.5 3.125 0.216506350946 3.21489929727e-09
0 2 1518.0 1860.5 321.64382475 0.000703580641306
0 3 74.0 18.0 5.24404424085 1.31389901518e-26
0 4 0.25 0.054 0.0843534231671 0.318014731127
0 5 1642.0 2036.0 460.890984941 0.000600650122819
1 0 222.0 200.8 49.30882274 0.0073764174633
1 1 4.5 4.4 1.59373774505 0.249826381181
1 2 1518.0 1251.2 103.971919286 0.000142604400687
1 3 74.0 98.0 20.9093280619 0.00987379173825
1 4 0.25 1.018 1.3654654884 0.249422651894
1 5 1642.0 1372.0 127.525683688 0.000332600736284
2 0 222.0 1301.0 420.403377722 3.52224687167e-05
2 1 4.5 32.5 10.3561575886 0.000996230932694
2 2 1518.0 1621.0 392.342325017 0.000982379194148
2 3 74.0 49.1 32.6975534253 0.00912991540072
2 4 0.25 1293.0 373.214415584 2.65215299121e-06
2 5 1642.0 832.85 829.922738874 0.000298854220172
[  1.24604662e-44   8.27916286e-14   7.67538782e-23]
[  1.50503939e-31   9.99999999e-01   9.27072935e-10]
